In [33]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time, os
os.chdir(os.path.dirname(os.getcwd()))
import json

In [ ]:
# Load the links we want to scrap
path = 'E:/_UChicago/MACS30122/final-project-ascending/'
os.chdir(path)
data = pd.read_csv('data/greatschools/schools_data_all_states.csv')
urls = data.loc[:, ['universal-id', 'overview-url']].dropna()

urls = ['https://www.greatschools.org/wisconsin/campbellsport/761-Wayne-Elementary-School/#Reviews']



In [28]:
# Make a request to the page and parse the html with BeautifulSoup

# Helper functions
def get_request(url):
    try:
        r = requests.get(url)
        if r.status_code == 200:
            return BeautifulSoup(r.text, 'html5lib')
        else:
            return None
    except:
        return None


In [55]:
def scrape_reviews_jsonld(url):
    """Scrape reviews from the JSON-LD <script> tag on a GreatSchools page."""
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error accessing {url}: {e}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    
# 1. Find the <script type="application/ld+json"> that contains the reviews
    script_tag = soup.find('script', attrs={'type': 'application/ld+json'})
    if not script_tag:
        print("No JSON-LD script tag found.")
        return NA

    # 2. Parse the JSON text
    try:
        data = json.loads(script_tag.string)
    except json.JSONDecodeError:
        print("Error decoding JSON from the <script> tag.")
        return []
    print(data)
    # 3. Extract reviews from data["review"], if present
    reviews_list = data.get('review', [])
    # all_reviews = []
    # for review_obj in reviews_list:
    #     # "reviewBody" is the text of the review
    #     review_text = review_obj.get('reviewBody', '').strip()
    #     all_reviews.append(review_text)

    reviews = []

    for review_obj in reviews_list:
        # "reviewBody" is the text of the review
        review = {}
        review['identity'] = review_obj.get('anthor', '').strip()
        review['date'] = review_obj.get('datePublished', '').strip()
        review['reviewbody'] = review_obj.get('reviewBody', '').strip()
        review['rating'] = None
        if 'reviewRating' in review_obj:
            review['rating'] = review_obj['reviewRating'].get('ratingValue', '').strip()
        review['detailedRating'] = review_obj.get('detailedRating', '').strip()
        # all_reviews.append(review_text)

        reviews.append(review)
    

    # return all_reviews
    return reviews